In [1]:
import pandas as pd

In [2]:
# Install a pip package in the current Jupyter kernel :)
import sys
!{sys.executable} -m pip install openpyxl

In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [378]:
csv_list = !ls 'sum'*'csv'

In [379]:
csv_list

['sum_adhd.csv',
 'sum_agg_beh.csv',
 'sum_bg.csv',
 'sum_echolalia.csv',
 'sum_gts_genes.csv',
 'sum_inv_mov.csv',
 'sum_neuro_dev.csv',
 'sum_ocd_beh.csv',
 'sum_patho.csv',
 'sum_self_mut.csv',
 'sum_tics.csv']

In [390]:
dfs = []
dataframes = []

for f in csv_list:
    dfs.append(pd.read_csv(f))
    dataframes.append(pd.read_csv(f))

### This is a script to open .csv generated by imdik-zekanowski-gts, to remove some characters, split SnipEff annotations and make an excel file with separate sheets

In [391]:
for idx, df in enumerate(dfs):
    
    df.replace({'\[|\]|\'':''}, regex = True, inplace = True) 
    df.replace({'\|':' '}, regex = True, inplace = True)  
    df.replace({'\^':' '}, regex = True,inplace = True)
   
    dataframes[idx] = df.SnpEff.str.split(" ", expand=True)
    dataframes[idx] = dataframes[idx].iloc[:, [1,2,7,10]]
    dataframes[idx].columns = ['variant_type','impact','location','AA']
    
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    df.drop(df.columns[0], axis = 1, inplace = True)

    dfs[idx] = pd.concat([df, dataframes[idx]], axis=1)
    
    d = dfs[0:(len(dfs)-1)]
    d = pd.concat(d)
    d = d[['sample','locus', 'genotype', 'alleles', 'family', 'sex', 'additional_pheno', 'Gene', 'impact']]
    d.sort_values(['family', 'sample'], axis=0, inplace=True, ascending=True)
    
dfs.append(pd.read_csv('GTS-coded.csv'))


with pd.ExcelWriter('rare_patients_only.xlsx') as writer:
    for idx, df in enumerate(dfs):
        try:
            df.to_excel(writer, sheet_name=str(csv_list[idx]))
        except IndexError:
            df.to_excel(writer, sheet_name='GTS-coded')
    d.to_excel(writer, sheet_name='sample_centric')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [477]:
# work on variants that occur > 1
df2 = pd.read_csv('high_double_all_samples.csv')

In [478]:
#find variants that occur in many many controls
df2[df2['disease'] != 'YES']['Gene'].value_counts()

['PRPF3']      78
['RAI1']       60
['TP53']       38
['GP1BA']      23
['TRPV3']      18
['ENTPD1']     16
['PCK1']       12
['TNK1']       10
['PRKRA']       7
['SCN4A']       7
['IDH2']        6
['HSPA9']       6
['TBP']         4
['HTT']         4
['TBCD']        4
['NDUFS7']      4
['GGT1']        4
['MTMR2']       3
['GMPPB']       3
['DMPK']        2
['CIC']         2
['FMO5']        2
['EEF2']        2
['SYNGAP1']     2
['CCDC40']      2
Name: Gene, dtype: int64

In [479]:
df2.replace({'\[|\]|\'':''}, regex = True, inplace = True) 
df2.replace({'\|':' '}, regex = True, inplace = True)  
df2.replace({'\^':' '}, regex = True,inplace = True)

In [480]:
df2 = df2[~df2['Gene'].isin(['PRPF3','RAI1','TP53','GP1BA','TRPV3','ENTPD1','PCK1','TNK1','PRKRA','SCN4A', 'IDH2', 'HSPA9','PAH'])]

In [481]:
df2 = df2[df2['disease'] != 'NO']

In [482]:
df = df2.SnpEff.str.split(" ", expand=True)
df = df.iloc[:, [1,2,7,10]]
df.columns = ['variant_type','impact','location','AA']

df2.drop(df2.columns[len(df2.columns)-1], axis=1, inplace=True)
df2.drop(df2.columns[0], axis = 1, inplace = True)

df2 = pd.concat([df2, df], axis=1)
df2.sort_values(['family', 'sample'], axis=0, inplace=True, ascending=True)

In [483]:
df2.to_csv('doubles_patients.csv')

### This is a script to open pickled objects, split some columns and create files for each family

In [9]:
cols = !ls ./py_objects/cols*
rows = !ls ./py_objects/rows*

In [325]:
info = []
tables = []

for col, row in zip(cols, rows):
    info.append(pd.read_pickle(col))
    tables.append(pd.read_pickle(row))

In [326]:
results = []

for idx, table in enumerate(tables):
    
    a = list(table.columns)

    b = [x for x in a if 'phased' not in x]
    b = [x for x in b if 'GQ' not in x]
    c = [x for x in b if 'ISEQ' in x]
    b = [x for x in b if 'ISEQ' not in x]
    b.remove('ANN')
    b.remove('PhastCons100way')
    d = b + c
    d.append('PhastCons100way')
    d.append('ANN')
    
    table = table[d]
    table = table.astype(str)
    
    table.replace({'\[|\]|\'':''}, regex = True, inplace = True) 
    table.replace({'\|':' '}, regex = True, inplace = True)  
    table.replace({'\^':' '}, regex = True,inplace = True)
    
    ann = table['ANN']
    ann = ann.str.split(" ", expand=True)
    ann = ann.iloc[:, [1,2,7,10]]
    ann.columns = ['variant_type','impact','location','AA']
    
    table.drop('ANN', axis=1, inplace=True)
    table = pd.concat([table, ann], axis=1)
    
    tables[idx] = table
    
    e = pd.DataFrame(table.columns).transpose()
    e.columns = table.columns
    f = pd.concat([e, table], axis=0)

    g = info[idx].transpose()
    h = pd.DataFrame(g.index)
    h.index = g.index
    i = pd.concat([h, g], axis = 1)

    i.columns = range(0,len(i.columns))
    i = i.reindex(columns = ['a']*3 + list(range(0,len(i.columns))) + ['a']*13)

    i.columns = f.columns
    i.index = range(0,len(i.index))

    f.index = range(len(i.index)+1,len(i.index)+len(f.index)+1)

    j = pd.concat([i, f], axis = 0)
    j.columns = range(0, len(j.columns))

    results.append(j)

    
    
with pd.ExcelWriter('families.xlsx') as writer:
    for idx, res in enumerate(results):
        res.to_excel(writer, sheet_name=str(res.iloc[1][4]), header=False, index=False)